In [41]:
%pip install faiss-cpu

# 1. Dense Vector Search (밀집 벡터 검색)


$$
\text{similarity}(q, d) = \cos(\theta)
= \frac{q \cdot d}{\|q\| \times \|d\|}
$$


- 원리 : 문장을 고차원 벡터로 변환하고 코사인 유사도를 이용해서 의미적 유사성 측정
- 장점
  - 의미적 유사성 포착(동의어)
  - 비가온다 <->장마가 시작됐다
- 단점
  - 고유명사, 숫자 등 정확한 매칭에 약해

# 2. BM25 (Best Match 25)
  - 키워드 기반 희소 검색

$$
\text{BM25}(D, Q) = \sum_{q_i \in Q} \text{IDF}(q_i) \cdot
\frac{f(q_i, D)(k_1 + 1)}
{f(q_i, D) + k_1\left(1 - b + b \cdot \frac{|D|}{\text{avgdl}}\right)}
$$


**IDF(qᵢ)**: 단어 qᵢ의 역문서 빈도 (드문 단어일수록 값이 큼)  
**f(qᵢ, D)**: 문서 D에서 단어 qᵢ가 등장한 횟수  
**k₁**: 포화 파라미터 (일반적으로 1.2 ~ 2.0)  
**b**: 문서 길이 정규화 파라미터 (일반적으로 0.75)  
**avgdl**: 전체 문서들의 평균 길이


- TF-IDF 개선버전, 문서 내 단어 빈도와 희소성을 고려
- 장점
  - 정확한 키워드 매칭
  - 고유명사 검색에 강함
- 단점
  - 의미적 유사성 포착 불가
  - 비<->장마 연결 어려움  

# 3. Reciprocal Rank Fusion (RRF)

$$
\text{RRF}(d) = \sum_i \frac{1}{k + \text{rank}_i(d)}
$$

**d**: 문서  
**k**: 상수 (일반적으로 60 사용)  
**rankᵢ(d)**: i번째 검색 시스템에서 문서 d가 받은 순위



- 여러검색 결과의 순위를 통합하여 최종 랭킹을 생성
- 장점
  - 다양한 검색 방식의 장점 결합
  - 점수 스케일이 다른 검색 결과도 통합 가능

In [42]:
# 문장 임베딩으로 벡터 검색
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

In [69]:
# 한국어 임베딩 모델 로드
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
# sample doc
documents = ["로버트 헨리 딕이 1946년에 연구했다", "2023년 AI 기술이 발전했다", "프린스턴 대학교 AI 연구소"]
# 문서 임베딩
doc_embeddings = model.encode(documents)

# FAISS 인덱스 생성
dimension =  doc_embeddings.shape[1]   #(3, 768)
index = faiss.IndexFlatL2(dimension)
index.add(doc_embeddings.astype('float32'))

# 쿼리 검색
query = '프린스턴 대학의 연구소'
query_embedding =  model.encode([query])
distance, indices =  index.search( query_embedding.astype('float32'), k=3)
print('검색결과')
for i, (dist,idx) in  enumerate( zip(distance[0], indices[0]) ):
  print(f' {i+1}. {documents[idx]} ( 거리 : {dist:.4f} ) 인덱스 : {indices[0]}')

검색결과
 1. 프린스턴 대학교 AI 연구소 ( 거리 : 226.3636 ) 인덱스 : [2 0 1]
 2. 로버트 헨리 딕이 1946년에 연구했다 ( 거리 : 388.5460 ) 인덱스 : [2 0 1]
 3. 2023년 AI 기술이 발전했다 ( 거리 : 698.7937 ) 인덱스 : [2 0 1]


BM25 검색 구현

In [44]:
import math
from collections import defaultdict
from transformers import AutoTokenizer
class SimpleBM25:
  def __init__(self, documents, k1=1.2, b=0.75):
    self.tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')
    self.k1 = k1
    self.b  = b
    self.documents = documents
    self.tokenized_docs = [ self.tokenizer.tokenize(doc) for doc in documents ]
    self.avg_doc_len = sum(len(d)  for d in self.tokenized_docs) / len(self.tokenized_docs)
    self.idf = self._compute_idf()
  def _compute_idf(self):
    idf = defaultdict(float)
    N = len(self.tokenized_docs)
    # TF 각 단어의 문서 빈도 계산
    tf = defaultdict(int)
    for doc in self.tokenized_docs:
      for token in set(doc):
        tf[token] += 1
    # IDF 계산
    for token,freq in tf.items():
      # 비율이 클수록(드문 토큰일수록) 값이 커지며, 흔한 토큰일 수록 작아진
      idf[token] = math.log((N - freq + 0.5) / (freq + 0.5)+1 )  # 토큰이 나타나지 않은 문서수  , 토큰이 등장한 문서수
    return idf
  def score(self, query):
    query_tokens =  self.tokenizer.tokenize(query)
    scores = []
    for doc in self.tokenized_docs:
      score = 0
      doc_len = len(doc)
      # 단어 빈도 계산
      tf = defaultdict(int)
      for token in doc:
        tf[token] += 1
      for token in query_tokens:
        if token in tf:
          freq = tf[token]
          numerator = self.idf[token] * freq + (self.k1 + 1)
          denoinator = freq + self.k1 * (1 - self.b + self.b*doc_len / self.avg_doc_len)
          score += numerator / denoinator
      scores.append(score)
    return scores



In [55]:
# 사용
documents = ["로버트 헨리 딕이 1946년에 연구했다", "2023년 AI 기술이 발전했다", "프린스턴 대학교 AI 연구소"]
bm25 = SimpleBM25(documents)
query = '프린스턴 대학의 연구소'
scores = bm25.score(query)
print(f'bm25 점수 : {scores}')

bm25 점수 : [0, 0, 5.057078766308966]


하이브리드 검색 : 밀집벡터 + bm25 -> rank
  - 여러 검색결과의 순위를 RRF로 통합

In [72]:
def reciprocal_rank_fusion(rankings, k=60):
  '''
  여러 검색결과의 순위를 RRf로 통합
  Args:
    rankings : 각 검색방식의 문서 인덱스 순위 리스트
    k: 상수
  Return:
    통합 점수로 정렬된(문서인덱스, 점수) 리스트
  '''
  print(f'rankings : {rankings}')
  rrf_scores = defaultdict(float)
  for ranking in rankings:   # rankings : [array([2, 0, 1]), array([2, 1, 0])]
    for rank, doc_id in enumerate(ranking, 1):
      rrf_scores[doc_id] += 1.0 / (k + rank)
  return sorted(rrf_scores.items(), key = lambda x : x[1], reverse=True)

def hybrid_search(query,dense_index, documents, bm25, top_k=3 ,rrf_k=60):
  # Dense Search FAISS
  query_embedding = model.encode([query])
  D,I =  dense_index.search(query_embedding.astype('float32'), k=top_k)
  desnse_ranking = np.array(I[0])
  print(f'desnse_ranking : {desnse_ranking}')

  # BM25
  sparse_scores =  bm25.score(query)
  sparse_ranking = np.array(np.argsort(sparse_scores)[::-1][:top_k] )
  print(f'sparse_ranking : {sparse_ranking}')

  # RRF 통합
  results = reciprocal_rank_fusion([desnse_ranking,sparse_ranking], k=rrf_k)
  return results

In [74]:
documents = ["로버트 헨리 딕이 1946년에 연구했다", "2023년 AI 기술이 발전했다", "프린스턴 대학교 AI 연구소"]
# 한국어 SBERT 모델
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
doc_embeddings = model.encode(documents).astype('float32')
dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(doc_embeddings)

bms25 = SimpleBM25(documents)

# 하이브리드 검색
query = '기술발전'
results = hybrid_search(query, index, documents,bm25,top_k=3)

print(f'하이브리드 검색 결과')
for rank, (doc_id, score) in enumerate(results, 1):
  try:
    print(f'{rank}. {documents[doc_id]} (점수 : {score:.4f})')
  except:
    pass

# desnse_ranking : [2 0 1]   2번문서가 1위
# sparse_ranking : [2 1 0]   2번 문서가 1위

desnse_ranking : [1 2 0]
sparse_ranking : [1 2 0]
rankings : [array([1, 2, 0]), array([1, 2, 0])]
하이브리드 검색 결과
1. 2023년 AI 기술이 발전했다 (점수 : 0.0328)
2. 프린스턴 대학교 AI 연구소 (점수 : 0.0323)
3. 로버트 헨리 딕이 1946년에 연구했다 (점수 : 0.0317)


In [48]:
results

[(np.int64(1), 0.03252247488101534),
 (np.int64(2), 0.032266458495966696),
 (np.int64(0), 0.03200204813108039)]